In [0]:
cd "E:\Text Mining\PROJECT"

[Errno 2] No such file or directory: 'E:\\Text Mining\\PROJECT'
/content


In [0]:
import pandas as p
import numpy as np
import itertools
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import re

In [0]:
def getCleanTweet(text):
    tok = WordPunctTokenizer()
    user_pattern = '@[A-Za-z0-9_]+'
    http_pattern = 'https?://[^ ]+'
    www_pattern = 'www.[^ ]+'
    combined_pattern = '|'.join((user_pattern, http_pattern, www_pattern))
    negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                    "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                    "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                    "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                    "mustn't":"must not"}
    neg_pattern = re.compile('\b(' + '|'.join(negations_dic.keys()) + ')\b')
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pattern, '', bom_removed)
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], stripped)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled).lower()
    cleaned = (" ".join(x for x in tok.tokenize(letters_only) if len(x) > 1)).strip()
    cleaned = ''.join(k + k if sum(1 for i in g) > 1 else k for k, g in itertools.groupby(cleaned))
    return cleaned

In [3]:
data = p.read_csv("tweet.csv", encoding="latin-1", names=["target", "ids", "date", "flag", "user", "text"])
data['clean_text'] = [getCleanTweet(text) for text in data['text']]
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   target      1600000 non-null  int64 
 1   ids         1600000 non-null  int64 
 2   date        1600000 non-null  object
 3   flag        1600000 non-null  object
 4   user        1600000 non-null  object
 5   text        1600000 non-null  object
 6   clean_text  1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [0]:
x_values = data['clean_text'].values
y_values = data['target'].values
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_values, y_values, test_size=0.01, random_state=0)

# Create 4 models - 
# tf unigram,  
# tf idf unigram, 
# tf unigram+bigram, 
# tf idf unigram+bigram
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf_unigram = CountVectorizer(encoding='latin-1',binary=True,min_df = 5 )
tf_idf_unigram = TfidfVectorizer(encoding='latin-1',use_idf=True,min_df = 5)
tf_uni_bi = CountVectorizer(encoding='latin-1',binary=False, ngram_range=(1,2),min_df = 5)
tf_idf_uni_bi = TfidfVectorizer(encoding='latin-1',use_idf=True, ngram_range=(1,2),min_df = 5)
tf_idf_bi = TfidfVectorizer(encoding='latin-1',use_idf=True, ngram_range=(2,2),min_df = 5)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer(encoding='latin-1',binary=False,min_df = 5 ,stop_words='english')
bi_vector = CountVectorizer(encoding='latin-1',stop_words='english',binary=False, ngram_range=(1,2),min_df = 5)

x_train_vec = vector.fit_transform(x_train)
print(x_train_vec.shape)

x_test_vec = vector.transform(x_test)
print(x_test_vec.shape)


(1120000, 40537)
(480000, 40537)


In [0]:
from sklearn.svm import LinearSVC as svm
svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec,y_train)
svm.score(x_test_vec, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7727020833333333

In [0]:
from sklearn.svm import LinearSVC as svm

In [9]:
#TF unigram

x_train_vec_tf = tf_unigram.fit_transform(x_train)
print(x_train_vec_tf.shape)

x_test_vec_tf = tf_unigram.transform(x_test)
print(x_test_vec_tf.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_tf,y_train)
svm.score(x_test_vec_tf, y_test)

(1584000, 49537)
(16000, 49537)


0.7991875

In [0]:
# tf idf unigram
from sklearn.svm import LinearSVC as svm
x_train_vec_idf = tf_idf_unigram.fit_transform(x_train)
print(x_train_vec_idf.shape)

x_test_vec_idf = tf_idf_unigram.transform(x_test)
print(x_test_vec_idf.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_idf,y_train)
svm.score(x_test_vec_idf, y_test)

(1120000, 40537)
(480000, 40537)


0.7736270833333333

In [0]:
# tf unigram_bi
from sklearn.svm import LinearSVC as svm
x_train_vec_tf_uni_bi= tf_uni_bi.fit_transform(x_train)
print(x_train_vec_tf_uni_bi.shape)

x_test_vec_tf_uni_bi = tf_uni_bi.transform(x_test)
print(x_test_vec_tf_uni_bi.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_tf_uni_bi,y_train)
svm.score(x_test_vec_tf_uni_bi, y_test)
    

(1120000, 185066)
(480000, 185066)


0.7677833333333334

In [0]:
# tf unigram_bi
from sklearn.svm import LinearSVC as svm
x_train_vec_idf_uni_bi= tf_idf_uni_bi.fit_transform(x_train)
print(x_train_vec_idf_uni_bi.shape)

x_test_vec_idf_uni_bi = tf_idf_uni_bi.transform(x_test)
print(x_test_vec_idf_uni_bi.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_idf_uni_bi,y_train)
svm.score(x_test_vec_idf_uni_bi, y_test) 

(1120000, 185066)
(480000, 185066)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7675333333333333

In [0]:
#TF unigram
from sklearn.svm import LinearSVC as svm
x_train_vec_tf = tf_unigram.fit_transform(x_train)
print(x_train_vec.shape)

x_test_vec_tf = tf_unigram.transform(x_test)
print(x_test_vec.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_tf,y_train)
print(svm.score(x_test_vec_tf, y_test))

# tf idf unigram
from sklearn.svm import LinearSVC as svm
x_train_vec_idf = tf_idf_unigram.fit_transform(x_train)
print(x_train_vec_idf.shape)

x_test_vec_idf = tf_idf_unigram.transform(x_test)
print(x_test_vec_idf.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_idf,y_train)
print(svm.score(x_test_vec_idf, y_test))

# tf unigram_bi
from sklearn.svm import LinearSVC as svm
x_train_vec_tf_uni_bi= tf_uni_bi.fit_transform(x_train)
print(x_train_vec_tf_uni_bi.shape)

x_test_vec_tf_uni_bi = tf_uni_bi.transform(x_test)
print(x_test_vec_tf_uni_bi.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_tf_uni_bi,y_train)
print(svm.score(x_test_vec_tf_uni_bi, y_test))

# tf unigram_bi
from sklearn.svm import LinearSVC as svm
x_train_vec_idf_uni_bi= tf_idf_uni_bi.fit_transform(x_train)
print(x_train_vec_idf_uni_bi.shape)

x_test_vec_idf_uni_bi = tf_idf_uni_bi.transform(x_test)
print(x_test_vec_idf_uni_bi.shape)

svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec_idf_uni_bi,y_train)
print(svm.score(x_test_vec_idf_uni_bi, y_test))
    

(1120000, 40537)
(480000, 40537)
0.79404375
(1120000, 40840)
(480000, 40840)
0.79404375
(1120000, 274975)
(480000, 274975)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7903125
(1120000, 274975)
(480000, 274975)
0.79016875


In [14]:
#tf_unigram
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import numpy as np
SVCpipe = Pipeline([('SVC',LinearSVC())])

from sklearn.svm import LinearSVC as svm
x_train_vec_tf = tf_unigram.fit_transform(x_train)
x_test_vec_tf = tf_unigram.transform(x_test)

# Gridsearch to determine the value of C
param_grid = {'SVC__C':(0.85,1,10)}
linearSVC = GridSearchCV(SVCpipe,param_grid,cv=3,return_train_score=True)
linearSVC.fit(x_train_vec_tf,y_train)
print(linearSVC.best_params_)
#linearSVC.coef_
#linearSVC.intercept_

bestlinearSVC = linearSVC.best_estimator_
bestlinearSVC.fit(x_train_vec_tf,y_train)
bestlinearSVC.coef_ = bestlinearSVC.named_steps['SVC'].coef_

print(bestlinearSVC.score(x_train_vec_tf,y_train))
print(bestlinearSVC.score(x_test_vec_tf,y_test))


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'SVC__C': 0.85}
0.812060606060606
0.7993125


In [10]:
#IDF-Unigram+Bigram
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import numpy as np
SVCpipe = Pipeline([('SVC',LinearSVC())])

from sklearn.svm import LinearSVC as svm
x_train_vec_idf_uni_bi = tf_idf_uni_bi.fit_transform(x_train)
x_test_vec_idf_uni_bi = tf_idf_uni_bi.transform(x_test)

# Gridsearch to determine the value of C
param_grid = {'SVC__C':(1,10)}
linearSVC = GridSearchCV(SVCpipe,param_grid,cv=3,return_train_score=True)
linearSVC.fit(x_train_vec_idf_uni_bi,y_train)
print(linearSVC.best_params_)

bestlinearSVC = linearSVC.best_estimator_
bestlinearSVC.fit(x_train_vec_idf_uni_bi,y_train)
bestlinearSVC.coef_ = bestlinearSVC.named_steps['SVC'].coef_
print(bestlinearSVC.score(x_train_vec_idf_uni_bi,y_train))
print(bestlinearSVC.score(x_test_vec_idf_uni_bi,y_test))
show_most_and_least_informative_features(tf_idf_uni_bi, svm, n=10)

{'SVC__C': 1}
0.8885277777777778
0.81625


In [13]:
#IDF Unigram
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import numpy as np
SVCpipe = Pipeline([('SVC',LinearSVC())])

from sklearn.svm import LinearSVC as svm
x_train_vec_idf = tf_idf_unigram.fit_transform(x_train)
x_test_vec_idf = tf_idf_unigram.transform(x_test)


# Gridsearch to determine the value of C
param_grid = {'SVC__C':(0.85,1)}
linearSVC = GridSearchCV(SVCpipe,param_grid,cv=3,return_train_score=True)
linearSVC.fit(x_train_vec_idf,y_train)
print(linearSVC.best_params_)
#linearSVC.coef_
#linearSVC.intercept_

bestlinearSVC = linearSVC.best_estimator_
bestlinearSVC.fit(x_train_vec_idf,y_train)
bestlinearSVC.coef_ = bestlinearSVC.named_steps['SVC'].coef_

print(bestlinearSVC.score(x_train_vec_idf,y_train))
print(bestlinearSVC.score(x_test_vec_idf,y_test))


{'SVC__C': 0.85}
0.8124444444444444
0.800375


In [12]:
#TF Unigram+Bigram
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import numpy as np
SVCpipe = Pipeline([('SVC',LinearSVC())])
from sklearn.svm import LinearSVC as svm

x_train_vec_tf_uni_bi = tf_uni_bi.fit_transform(x_train)
x_test_vec_tf_uni_bi = tf_uni_bi.transform(x_test)

# Gridsearch to determine the value of C
param_grid = {'SVC__C':(1,10)}
linearSVC = GridSearchCV(SVCpipe,param_grid,cv=3,return_train_score=True)
linearSVC.fit(x_train_vec_tf_uni_bi,y_train)
print(linearSVC.best_params_)
#linearSVC.coef_
#linearSVC.intercept_

bestlinearSVC = linearSVC.best_estimator_
bestlinearSVC.fit(x_train_vec_tf_uni_bi,y_train)
bestlinearSVC.coef_ = bestlinearSVC.named_steps['SVC'].coef_

print(bestlinearSVC.score(x_train_vec_tf_uni_bi,y_train))
print(bestlinearSVC.score(x_test_vec_tf_uni_bi,y_test))


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'SVC__C': 1}
0.9066357323232324
0.794
